# Reproduction of "ALBERTA’S FISCAL RESPONSES TO FLUCTUATIONS IN NON-RENEWABLE-RESOURCE REVENUE" in python

This notebook attempts to replicate the empirical results, tables, and figures produced in the paper by Ergete Ferede, published by the University of Calgary school of public policy in Volume 11:24, September 2018.
The original paper is here: [https://www.policyschool.ca/wp-content/uploads/2018/09/NRR-Ferede.pdf](https://www.policyschool.ca/wp-content/uploads/2018/09/NRR-Ferede.pdf)

## Setup and data acquisition

This section of the code loads required modules, downloads the required data sets, and reads them into DataFrames

In [1]:
from pathlib import Path
import datetime as dt
import requests
import pandas as pd
import stats_can

In [65]:
def download_data():
    """Download the excel file for the analysis from the policy school page
    
    Note the readme sheet on the first file. Credit to Kneebone and Wilkins for
    assembling it, and policy school for hosting it
    
    Returns
    -------
    fname: pathlib.Path
        A path object with the location and name of the data
    """
    print('Downloading data set')
#     url = 'https://www.policyschool.ca/wp-content/uploads/2019/01/Provincial-Government-Budget-Data-January-2019FINAL-USE.xlsx'
    url = 'https://www.policyschool.ca/wp-content/uploads/2019/03/Provincial-Government-Budget-Data-March-2019.xlsx'
    response = requests.get(
        url,
        stream=True,
        headers={'user-agent': None}
    )
    fname = Path('.').joinpath('data').joinpath('budgets.xlsx')
    with open (fname, 'wb') as outfile:
        for chunk in response.iter_content(chunk_size=512):
            if chunk: # filter out keep-alive new chunks
                outfile.write(chunk)
    return fname

def get_file(force_update=False):
    """Return the data set, download if necessary"""
    fname = Path('.').joinpath('data').joinpath('budgets.xlsx')
    if not fname.exists() or force_update:
        download_data()
    return fname


def get_date_index(df):
    """Helper function to turn budget year strings into datetimes"""
    date_index = pd.to_datetime(
        df
        .assign(year=lambda df: df['budget_yr'].str[0:4].astype(int))
        .assign(month=1)
        .assign(day=1)
        [['year', 'month', 'day']]
    )
    return date_index


def read_ab_budget():
    """Read Alberta budget data 
    
    Downloads the data if necessary, reads it in and gives
    the variables easier to work with names
    
    Returns
    -------
    df: pandas.DataFrame
        Alberta's revenue and expenditure tables
    """
    fname = get_file()
    df = (
        pd.read_excel(
            fname,
            sheet_name='Alberta',
            header=3,
            index_col=1,
            skipfooter=21
        )
        # Because of the merged cells we get an empty first row
        .loc[lambda x: x.index.notnull()]
        # Not sure where the empty first column comes from but drop it
        .drop(columns='Unnamed: 0')
        .reset_index()
        .rename(columns={
            'index': 'budget_yr',
            'Personal Income Tax': 'personal_income_tax',
            'Corporation Income Tax': 'corporate_income_tax',
            'Retail Sales Tax': 'retail_sales_tax',
            'Federal Cash Transfers': 'federal_cash_transfers',
            'Natural Resource Revenue': 'natural_resource_revenue',
            'Other Own-Source Revenue': 'other_own_source_revenue',
            'Total Revenue': 'total_revenue',
            'Health': 'health_exp',
            'Social Services': 'social_services_exp',
            'Education': 'education_exp',
            'Other Program Expenditures': 'other_program_exp',
            'Total Program Expenditures': 'total_prog_exp',
            'Debt Service': 'debt_service',
            'Total  Expenditures': 'total_exp',
            'Unnamed: 16': 'annual_deficit'
        })
        .assign(budget_dt=lambda df: get_date_index(df))
        .set_index('budget_dt')
    )  
    return df

def read_heritage():
    """Heritage Trust fund deposits are separate"""
    fname = get_file()
    df = (
        pd.read_excel(
            fname,
            sheet_name='Alberta',
            header=None,
            usecols='D:G',
            names=['budget_yr', 'resource_allocation', 'deposits', 'advance_edu'],
            skiprows=71,
            skipfooter=1
        )
        .loc[lambda df: ~df['budget_yr'].isna()]
        .set_index('budget_yr')
        .fillna(0)
        .assign(total_heritage=lambda df: df.sum(axis='columns'))
        # Add a dummy variable to indicate heritage fund deposit years
        .assign(heritage_dummy=1)
        .reset_index()
        .assign(budget_dt=lambda df: get_date_index(df))
        .drop(columns='budget_yr')
        .set_index('budget_dt')
    )
    return df


def clean_budget():
    """Combine base budget with heritage deposits"""
    budg = read_ab_budget()
    heritage= read_heritage()
    budg_clean = (
        budg
        .assign(other_revenue=lambda df: df[['retail_sales_tax', 'federal_cash_transfers', 'other_own_source_revenue']].sum(axis='columns'))
        .reindex(columns=['personal_income_tax', 'corporate_income_tax', 'natural_resource_revenue', 'other_revenue', 'total_prog_exp', 'debt_service'])
        .merge(heritage[['total_heritage', 'heritage_dummy']], how='left', left_index=True, right_index=True)
        .fillna(0)
        .assign(natural_resource_revenue=lambda df: df['natural_resource_revenue'] - df['total_heritage'])
        .assign(total_revenue=lambda df: df[['personal_income_tax', 'corporate_income_tax', 'natural_resource_revenue', 'other_revenue']].sum(axis='columns'))
        .assign(total_expenditure=lambda df: df[['total_prog_exp', 'debt_service']].sum(axis='columns'))
        .assign(deficit=lambda df: df['total_expenditure'] - df['total_revenue'])
        .drop(columns=['total_revenue', 'total_expenditure', 'total_heritage'])
        .astype('float64')
    )
    return budg_clean


def per_capita_data():
    """Read in population data to calculate per capita estimates
    
    Rolling mean indexed on January year N+1 is the March to March
    average population for fiscal year N
    Applying a date offset of -1 year and taking only
    January data of these rolling means gives us a population average
    on the same basis as the fiscal tables
    """
    table = '17-10-0009-01'
    df = (
        stats_can.table_to_df(table, path='data')
        .loc[lambda x: x['GEO'] == 'Alberta']
        .loc[lambda x: x['REF_DATE'] >= '1965']
        .set_index('REF_DATE')
        [['VALUE']]
        .rename(columns={'VALUE':'population'})
        .rolling(4, closed='left')
        .mean()
        .reset_index()
        .assign(budget_dt=lambda df: df['REF_DATE'] - pd.DateOffset(years=1))
        .loc[lambda x: x['budget_dt'].dt.year >= 1965]
        .loc[lambda x: x['budget_dt'].dt.month == 1]
        .drop(columns='REF_DATE')
        .set_index('budget_dt')
        .copy()
    )
    return df


def inflation_data():
    vecs = ('v41692327', 'v41690973')
    df = (
        stats_can.vectors_to_df_local(vecs, path='data', start_date=dt.date(1965, 1, 1))
        .rename(columns={'v41692327': 'ab_inflation', 'v41690973': 'ca_inflation'})
    )
    mask = df['ab_inflation'].isna()
    df.loc[mask, 'ab_inflation'] = df.loc[mask, 'ca_inflation']
    df = (
        df
        .drop(columns='ca_inflation')
        .rolling(4, closed='left')
        .mean()
        .reset_index()
        .assign(budget_dt=lambda df: df['REF_DATE'] - pd.DateOffset(years=1))
        .loc[lambda x: x['budget_dt'].dt.year >= 1965]
        .loc[lambda x: x['budget_dt'].dt.month == 1]
        .drop(columns='REF_DATE')
        .set_index('budget_dt')
        .copy()
    )
    # Rescale to 2017 = 100 (this is fiscal year 2017, original may have done calendar year)
    inf_2017 = float(df.loc['2017', 'ab_inflation'])
    df = df / inf_2017
    return df


def budget_real_per_capita():
    clean_budget_df = clean_budget()
    scale_cols = clean_budget_df.columns.drop('heritage_dummy')
    per_capita = per_capita_data()
    inflation = inflation_data()
    df = (
        clean_budget()
        # Convert to millions of dollars
        .mul(1000000)
        .div(per_capita, axis=scale_cols)
        .div(inflation, axis=scale_cols)
    )
    return df

## Specification and data



In [66]:
budget_real_per_capita()

,ab_inflation,corporate_income_tax,debt_service,deficit,heritage_dummy,natural_resource_revenue,other_revenue,personal_income_tax,population,total_prog_exp
budget_dt,,,,,,,,,,
1965-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1966-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1967-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1968-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1969-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1971-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1972-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
